# import

In [1]:
from datasets import load_dataset
import pandas as pd
import torch
from torch.utils.data import Dataset
from underthesea import word_tokenize
from pyvi import ViTokenizer
import os
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import unidecode
import re
import random
from bs4 import BeautifulSoup
import contractions
import pandas as pd
import torch
from pathlib import Path
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from torchtext.data.metrics import bleu_score
from torchmetrics import Recall, Precision, FBetaScore, Accuracy
import numpy as np

/home/phamngocthi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/phamngocthi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# config

In [2]:
raw_data_path = "./dataset/train.csv"

tokenizer_noise_vi_path = "./dataset/tokenizer_noise_vi.json"
tokenizer_vi_path = "./dataset/tokenizer_vi.json"

noise_vi_dataset_path = "./dataset/noise_vi.csv"
vi_dataset_path = "./dataset/vi.csv"

train_noise_vi_path = "./dataset/train_noise_vi.csv"
train_vi_path = "./dataset/train_vi.csv"

val_noise_vi_path = "./dataset/val_noise_vi.csv"
val_vi_path = "./dataset/val_vi.csv"

test_noise_vi_path = "./dataset/test_noise_vi.csv"
test_vi_path = "./dataset/test_vi.csv"

val_ratio = 0.1
test_ratio = 0.01
num_split_item = 150000
num_val = int(num_split_item * val_ratio)
num_test = int(num_split_item * test_ratio)

vocab_size = 20000
context_size = 2

# read dataset

In [3]:
raw_data = pd.read_csv(raw_data_path)
raw_data

,noise_vi,vi
0,họ có mối quan hệ san xuất chặt chẽ phi thường...,họ có mối quan hệ sản xuất chặt chẽ phi thường...
1,họ đặt mình vào máy chụp nao bộ -- có thể cách...,họ đặt mình vào máy chụp não bộ -- có thể cách...
2,"đây là bài mo đầu cho brian ẽno , sự cảm lhậl ...","đây là bài mở đầu cho brian eno , sự cảm nhận ..."
3,hàng hoá là nen tảng của nền kinh tế nông nghi...,hàng hoá là nền tảng của nền kinh tế nông nghi...
4,không một quốc gia mv latinh nào khác có nhiều...,không một quốc gia mỹ latinh nào khác có nhiều...
...,...,...
424692,còn đó chính xác những gĩ mà người mĩ yêu cầu .,còn đó chính xác những gì mà người mỹ yêu cầu .
424693,"tôi sệ nói , chên hết chính là nạn buôn lậu ng...","tôi sẽ nói , trên hết chính là nạn buôn lậu ng..."
424694,trên thiên đường bạl thấy những người thuê đôi...,trên thiên đường bạn thấy những người thuê đôi...
424695,"vâng , moi người đã nêu lên ý kiến của mình tr...","vâng , mỗi người đã nêu lên ý kiến của mình tr..."


# split and save noise_vi and vi dataset

In [4]:
# split noise_vi and vi column
noise_vi_dataset = {"text": raw_data['noise_vi']}
vi_dataset = { "text": raw_data['vi']}

noise_vi_dataset = pd.DataFrame(noise_vi_dataset)
vi_dataset = pd.DataFrame(vi_dataset)

# handle word with context size

In [5]:
def handle_word_with_context_size(token_ids, context_size):
    targets = []
    contexts = []

    for i in range(context_size, len(token_ids) - context_size):
        target_word = [token_ids[i]]
        context_words = [token_ids[j] for j in range(i - context_size, i)] + \
                        [token_ids[j] for j in range(i + 1, i + context_size + 1)]
        targets.append(target_word)
        contexts.append(context_words)

    return targets, contexts

# define custom dataset

In [6]:
# define custom dataset

class Word2VecDataset(Dataset):
    def __init__(self, dataset, tokenizer, context_size):
        self.context_size = context_size
        self.tokenizer = tokenizer
        self.targets = []
        self.contexts = []
        for text in dataset:
            token_ids = self.tokenizer.encode(text).ids
            target, context = handle_word_with_context_size(token_ids, self.context_size)
            self.targets += target
            self.contexts += context

    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return torch.tensor(self.targets[idx]), torch.tensor(self.contexts[idx])

# dataset split 150000 item and save

In [7]:
# split train, val, test dataset with ratio val and test
def split_train_val_test(dataset, num_val, num_test):
    # shuffle dataset
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    dataset = dataset[:num_split_item]
    # dataset is DataFrame with 1 column is text
    num_train = num_split_item - num_val - num_test
    train_dataset = dataset[:num_train]
    val_dataset = dataset[num_train:num_train + num_val]
    test_dataset = dataset[(num_train + num_val):]
    return train_dataset, val_dataset, test_dataset

train_vi, val_vi, test_vi = split_train_val_test(vi_dataset, num_val, num_test)
train_noise_vi, val_noise_vi, test_noise_vi = split_train_val_test(noise_vi_dataset, num_val, num_test)

# save all
train_noise_vi.to_csv(train_noise_vi_path, index=False)
train_vi.to_csv(train_vi_path, index=False)

val_noise_vi.to_csv(val_noise_vi_path, index=False)
val_vi.to_csv(val_vi_path, index=False)

test_noise_vi.to_csv(test_noise_vi_path, index=False)
test_vi.to_csv(test_vi_path, index=False)

# load all
train_noise_vi = pd.read_csv(train_noise_vi_path)
train_vi = pd.read_csv(train_vi_path)

val_noise_vi = pd.read_csv(val_noise_vi_path)
val_vi = pd.read_csv(val_vi_path)

test_noise_vi = pd.read_csv(test_noise_vi_path)
test_vi = pd.read_csv(test_vi_path)

# check all
print("train_noise_vi", train_noise_vi.head())
print("train_vi", train_vi.head())
print("======================================")

print("val_noise_vi", val_noise_vi.head())
print("val_vi", val_vi.head())
print("======================================")

print("test_noise_vi", test_noise_vi.head())
print("test_vi", test_vi.head())
print("======================================")

train_noise_vi                                                 text
0        nhưng nó không phải là kết luận bị bỏ qua .
1  đó lạ phản ứng , thái độ trước sự đàn áp tôi đ...
2  thế giới của anh lầ của chung tôi . mt : như t...
3  " chôn mìlh sao ? " một trong những ví dụ hiếm...
4  con giật đặc biệt này bơi vỏng quanh trong suố...
train_vi                                                 text
0  và tôi nói , " vâng , em cũng vậy -- nhưng em ...
1  với tôi đây là ví dụ tột đỉnh về sức mạnh gây ...
2  đoạn văn thứ 2 , bạn có thể phải thực thành th...
3  ông ấy đã không bao giờ tin tôi , nhưng tôi kh...
4  đây nè , đây mới chính là sự sáng tạo thực sự ...
val_noise_vi                                                 text
0  lhâl dna củẳ bắt đầu tôn hơp những protein mới...
1  là thước đo vĩnh vĩn . một thước đo của chúng ...
2  bợi vậy tôi đang tham gia các hoạt độg để tăng...
3  và bạl thẻ nghĩ côg việc đó giống như vịc chún...
4  vì vậy , nàm cha thuc chất là nỗ lưc của một n...
val_vi   

# create tokenizer

In [8]:
def get_all_sents(dataset):
    for item in dataset:
        yield item

def get_tokenizer(dataset, tokenizer_path):
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()

    trainer = WordLevelTrainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    tokenizer.train_from_iterator(get_all_sents(dataset), trainer)
    tokenizer.save(tokenizer_path)
    return tokenizer

In [9]:
tokenizer_noise_vi = get_tokenizer(train_noise_vi["text"], tokenizer_noise_vi_path)
tokenizer_vi = get_tokenizer(train_vi["text"], tokenizer_vi_path)
print(tokenizer_noise_vi.get_vocab_size())
print(tokenizer_vi.get_vocab_size())

20000
12917


In [10]:
noise_vi_word2vec_dataset = Word2VecDataset(train_noise_vi["text"], tokenizer_noise_vi, context_size)
vi_word2vec_dataset = Word2VecDataset(train_vi["text"], tokenizer_vi, context_size)

# create dataloader

In [11]:
# create dataloader
batch_size = 64
noise_vi_dataloader = DataLoader(noise_vi_word2vec_dataset, batch_size=batch_size, shuffle=True)
vi_dataloader = DataLoader(vi_word2vec_dataset, batch_size=batch_size, shuffle=True)

In [12]:
# check noise_vi and vi dataloader
for targets, contexts in noise_vi_dataloader:
    print(targets.shape, contexts.shape)
    break

for targets, contexts in vi_dataloader:
    print(targets.shape, contexts.shape)
    break

torch.Size([64, 1]) torch.Size([64, 4])
torch.Size([64, 1]) torch.Size([64, 4])
